In [ ]:





# from scipy.sparse import csr_matrix


from scipy.sparse import hstack
import eli5
import calendar

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                               
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

### Загрузка входных данных

#### Загрузка

In [ ]:
# !unzip capstone_user_identification.zip

In [2]:
# from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import pickle

In [17]:
# PATH_TO_DATA = '../capstone_user_identification'
PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

times = ['time%s' % i for i in range(1, 11)]
# sites = ['site%s' % i for i in range(1, 11)]
df_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=times)
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=times)
df_train.shape, df_test.shape

((253561, 21), (82797, 20))

In [18]:
df_train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  date

In [20]:
df_train.sort_values(by='time1', inplace=True)

Словарь сайтов

In [21]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as f:
    site2id  = pickle.load(f)
id2site = {v:k for (k, v) in site2id.items()}
id2site[0] = 'unknown'

### Обработка

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, TfidfVectorizer

import numpy as np

In [34]:
class DataPreparator(BaseEstimator, TransformerMixin):
    """
    Fill NaN with zero values;
    Prepare a (Count)Vectorizer friendly 2D-list from data;
    Site ids to names.
    """
    def __init__(self, id2site):
        super().__init__()
        self.id2site = id2site
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        # Convert dataframe rows to strings
        return X[sites].fillna(0).astype('int').apply(lambda row: ' '.join([id2site[i] for i in row]), axis=1).tolist()

In [25]:
# train_df_sites = train_df[sites].fillna(0).astype('int').apply(lambda row: ' '.join([id2site[i] for i in row]), axis=1).tolist()
# test_df_sites = test_df[sites].fillna(0).astype('int').apply(lambda row: ' '.join([id2site[i] for i in row]), axis=1).tolist()

In [ ]:
# vectorizer_params = {
#     'ngram_range': (1, 5),
#     'max_features': 50000,
#     'tokenizer': lambda s: s.split()
# }

### Feature engineering

In [26]:
class AttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """ 
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        # intraday features
        hour = X['time1'].apply(lambda ts: ts.hour)
        morning = ((hour >= 7) & (hour <= 11)).astype('int')
        day = ((hour >= 12) & (hour <= 18)).astype('int')
        evening = ((hour >= 19) & (hour <= 23)).astype('int')
        
        # season features
        month = X['time1'].apply(lambda ts: ts.month)
        summer = ((month >= 6) & (month <= 8)).astype('int')
        
        # day of the week features
        weekday = X['time1'].apply(lambda ts: ts.weekday()).astype('int')
        
        # year features
        year = X['time1'].apply(lambda ts: ts.year).astype('int')
        
        X = np.c_[morning.values, day.values, evening.values, summer.values, weekday.values, year.values]
        return X

In [27]:
class ScaledAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new features, that should be scaled.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # session time features
        times = ['time%s' % i for i in range(1, 11)]
        # session duration: take to the power of 1/5 to normalize the distribution
        session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) ** 0.2
        # number of sites visited in a session
        number_of_sites = X[times].isnull().sum(axis=1).apply(lambda x: 10 - x)
        # average time spent on one site during a session
        time_per_site = (session_duration / number_of_sites) ** 0.2
        
        X = np.c_[session_duration.values]
        return X

In [36]:
vectorizer_pipeline = Pipeline([
    ("preparator", DataPreparator(id2site)),
#     ("list_preparator", ListPreparator()),
    ("vectorizer", CountVectorizer(ngram_range=(1, 3), max_features=50000))
])

attributes_pipeline = Pipeline([
    ("adder", AttributesAdder())
])

scaled_attributes_pipeline = Pipeline([
    ("adder", ScaledAttributesAdder()),
    ("scaler", StandardScaler())
])

In [ ]:
# %%time
# vectorizer = TfidfVectorizer(**vectorizer_params)
# # vectorizer = CountVectorizer(**vectorizer_params)
# X_train_sparse = vectorizer.fit_transform(train_df_sites)
# X_test_sparse = vectorizer.transform(test_df_sites)
# y = train_df['target']
# X_train_sparse.shape, X_test_sparse.shape

In [37]:
full_pipeline = FeatureUnion(transformer_list=[
('vectorizer_pipeline', vectorizer_pipeline),
('attributes_pipeline', attributes_pipeline),
('scaled_attributes_pipeline', scaled_attributes_pipeline)
])

In [38]:
%%time
X_train = full_pipeline.fit_transform(df_train)
X_test = full_pipeline.transform(df_test)

y_train = df_train["target"].astype('int').values

CPU times: user 50.1 s, sys: 403 ms, total: 50.5 s
Wall time: 50.5 s


In [39]:
X_train

<253561x50007 sparse matrix of type '<class 'numpy.float64'>'
	with 11589023 stored elements in Compressed Sparse Row format>

In [ ]:
# scaler = StandardScaler()
# duration_train = (train_df[times].max(axis=1) - train_df[times].min(axis=1))\
#                     .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
# # number_of_sites = train_df[times].isnull().sum(axis=1).apply(lambda x: 10 - x).values.reshape(-1, 1)
# # time_per_site = (duration_train / number_of_sites)
# # scaler.fit(np.concatenate((duration_train, time_per_site), axis=1))
# scaler.fit(duration_train)

In [ ]:
def features(df):
    # intraday features
    hour = df['time1'].apply(lambda ts: ts.hour).values
    morning = ((hour >= 7) & (hour <= 11)).astype('int').reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').reshape(-1, 1)
#     night = ((hour >= 0) & (hour <= 6)).astype('int').reshape(-1, 1)
    
    # week features
    weekday = df.time1.apply(lambda ts: ts.weekday()).values.reshape(-1, 1)
#     is_monday = np.isin(weekday, [0])
#     is_wednesday = np.isin(weekday, [2])
#     is_sunday = np.isin(weekday, [6])
#     is_weekend = np.isin(weekday, [0, 2, 6])
#     weekdays = pd.get_dummies(weekday.flatten()).to_numpy().reshape(7, -1, 1)
    
    # month features
    month = df.time1.apply(lambda ts: ts.month).values.reshape(-1, 1)

    # year features
    year = df['time1'].apply(lambda t: 100 * t.year).values.reshape(-1, 1) / 1e5
    year_month = df['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5

    # session features
    duration = (df[times].max(axis=1) - df[times].min(axis=1))\
                    .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
#     unique_sites = df[sites].apply(lambda sites: sites.nunique(), axis=1).values.reshape(-1, 1)
#     number_of_sites = df[times].isnull().sum(axis=1).apply(lambda x: 10 - x).values.reshape(-1, 1)
#     time_per_site = (duration / number_of_sites)
    duration = scaler.transform(duration)
#     scaled = scaler.transform(np.concatenate((duration, time_per_site), axis=1)).reshape(2, -1, 1)
    
    # 
    feature_names =  ['morning', 'day', 'evening', 'year', 'year_month', 'weekday', 'month', 
                      'duration'] # + list(calendar.day_name)
    result = np.array([morning, day, evening, year, year_month, weekday, month, duration]) # is_wednesday, is_monday, is_sunday
#     result = np.concatenate((result, scaled), axis=0)
    
    return result, feature_names

In [ ]:
# %%time
# X_train_features, feature_names = features(train_df)
# X_test_features, _ = features(test_df)
# X_train = hstack([X_train_sparse, *X_train_features])
# X_test = hstack([X_test_sparse, *X_test_features])

In [ ]:
pd.DataFrame(np.moveaxis(X_train_features.squeeze(2), 1, 0), columns=feature_names)

### Модель

In [44]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

#### Обучение и подбор гиперпараметров

In [42]:
model = SGDClassifier(loss='log', random_state=17, n_jobs=-1)

In [43]:
grid = {
    'penalty': ['l2'],
    'alpha': np.linspace(3e-05, 5e-05, 11)
}

In [46]:
%%time
time_split = TimeSeriesSplit(n_splits=10)

search = GridSearchCV(model, param_grid=grid, cv=time_split, scoring='roc_auc')
search.fit(X_train, y_train)

CPU times: user 6min 13s, sys: 1.07 s, total: 6min 14s
Wall time: 6min 11s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             estimator=SGDClassifier(loss='log', n_jobs=-1, random_state=17),
             param_grid={'alpha': array([3.0e-05, 3.2e-05, 3.4e-05, 3.6e-05, 3.8e-05, 4.0e-05, 4.2e-05,
       4.4e-05, 4.6e-05, 4.8e-05, 5.0e-05]),
                         'penalty': ['l2']},
             scoring='roc_auc')

In [48]:
search.best_params_, search.best_score_

({'alpha': 5e-05, 'penalty': 'l2'}, 0.7527309434569364)

In [49]:
%%time

cv_scores = cross_val_score(search.best_estimator_, X_train, y_train, cv=time_split, scoring='roc_auc', n_jobs=-1)

CPU times: user 115 ms, sys: 452 ms, total: 567 ms
Wall time: 19.3 s


In [50]:
cv_scores, cv_scores.mean(), cv_scores.std()

(array([0.64165304, 0.53717647, 0.75649164, 0.6991099 , 0.73827052,
        0.80245029, 0.78631315, 0.86397755, 0.79459934, 0.90726753]),
 0.7527309434569364,
 0.10174015782448255)

0.9315069358401142,
 0.044345068769381366 0 2 6

0.9180564468510812,
 0.05889198351691508 baseline

0.9232962337585114,
 0.058257317090901546

0.9287111177172779,
 0.04434090003756869 2

In [ ]:
search.best_estimator_.fit(X_train, y)
logit_test_pred_proba = search.best_estimator_.predict_proba(X_test)

In [ ]:
pd.DataFrame({'feature': feature_names, 'coef': search.best_estimator_.coef_.flatten()[-len(feature_names):]})

In [ ]:
eli5.show_weights(estimator=search.best_estimator_, feature_names=vectorizer.get_feature_names() + feature_names, top=30)

In [ ]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'result.csv')